# Setup

In [1]:
import sys

sys.path.append("..")
import torch as th
_ = th.set_grad_enabled(False)

# Offline Dashboard

**Note that if you plan to only run the online dashboard, you might want to switch to a non-gpu runtime.**

In [2]:
from tools.cc_utils import get_available_models
from IPython.display import Markdown
models = get_available_models()
# Display available CrossCoder models
models_md = "\n".join([f"- {model}" for model in models])
display(Markdown(f"## Available CrossCoders:\n{models_md}"))

## Available CrossCoders:
- gemma-2-2b-crosscoder-l13-mu4.1e-02-lr1e-04
- gemma-2-2b-L13-mu5.2e-02-lr1e-04-2x100M-local-shuffling-SAELoss
- gemma-2-2b-L13-k100-lr1e-04-local-shuffling-CCLoss
- gemma-2-2b-L13-k100-lr1e-04-local-shuffling-SAELoss

In [3]:
crosscoder = "gemma-2-2b-L13-k100-lr1e-04-local-shuffling-CCLoss"  # the topk from the paper

⚠️ Norm diff is from 0 = chat only to 1 = base only which is the opposite of the paper ⚠️

In [4]:
from tools.utils import load_latent_df
df = load_latent_df(crosscoder)
df.sort_values(by='dec_norm_diff').head(10)

,tag,dec_norm_diff,lmsys_ctrl_%,lmsys_bos_%,lmsys_user_%,lmsys_assistant_%,lmsys_freq,lmsys_ctrl_freq,lmsys_non_ctrl_freq,dec_cos_sim,...,beta_error_base,beta_error_chat,beta_ratio_error,beta_reconstruction_base,beta_reconstruction_chat,beta_ratio_reconstruction,beta_activation_base,beta_activation_chat,beta_activation_ratio,max_act_val
latent,,,,,,,,,,,,,,,,,,,,,
62019,Chat only,0.047370,0.475136,0.0,0.203070,0.796930,0.008673,0.075924,0.004814,-0.200589,...,-0.020656,0.968877,-0.021320,0.585335,2.631238,0.222456,0.583684,2.600111,0.224484,64.373642
58070,Chat only,0.050693,0.248430,0.0,0.073774,0.926226,0.008763,0.040109,0.006964,0.081631,...,0.009499,0.985372,0.009640,0.863005,2.271316,0.379958,0.864228,2.256692,0.382962,107.302834
47844,Chat only,0.054111,0.490756,0.0,0.263789,0.736211,0.005271,0.047662,0.002839,-0.263505,...,-0.033485,1.010460,-0.033139,0.348303,2.245054,0.155142,0.343336,2.255514,0.152221,100.707672
35811,Chat only,0.060675,0.698904,0.0,0.505556,0.494444,0.001326,0.017077,0.000422,-0.211076,...,-0.034815,1.027078,-0.033897,0.013242,1.777351,0.007451,0.004042,1.804431,0.002240,95.252167
54087,Chat only,0.060918,0.586826,0.0,0.615485,0.384515,0.001609,0.017395,0.000703,-0.016421,...,-0.004785,0.925825,-0.005168,0.298635,2.116537,0.141096,0.295852,2.042361,0.144858,82.350746
50586,Chat only,0.061734,0.683073,0.0,0.283221,0.716779,0.003567,0.044885,0.001195,-0.235062,...,-0.043001,0.996189,-0.043165,0.816356,2.965883,0.275249,0.802378,2.962077,0.270883,76.740273
70149,Chat only,0.064321,0.269654,0.0,0.174622,0.825378,0.003078,0.015293,0.002377,-0.105011,...,-0.014167,1.039407,-0.013630,0.558428,2.540966,0.219770,0.557770,2.580371,0.216159,89.759666
63570,Chat only,0.064870,0.585160,0.0,0.697555,0.302445,0.002183,0.023535,0.000958,0.141863,...,0.009393,1.013318,0.009269,0.571538,3.214335,0.177809,0.562524,3.227657,0.174282,101.008492
37314,Chat only,0.065291,0.722848,0.0,0.881730,0.118270,0.001565,0.020836,0.000459,-0.023131,...,-0.012094,1.003398,-0.012053,0.249597,1.778328,0.140355,0.240524,1.781727,0.134995,75.424850


### Cool features to explore
Explore some features from the df above, or check the features we already manually analyzed [here](https://flax-group-6cc.notion.site/Using-CrossCoder-for-model-diffing-e9e3e6d48cc542a8b594ab737936d433?pvs=25#e917941297da40278d8c38d99fca9325)

A quick recap is available in the cell below

- **70149**: Refusal related latent: Requests for harmful instructions.
- **7736**: Refusal related latent: Generally sensitive content.
- **24613**: Refusal related latent: Unethical content relating to race, gender and stereotypes.
- **20384**: Refusal related latent: Requests for harmful instructions.
- **38009**: Refusal related latent: The model has refused to answer a user input.
- **2138**: Personal questions: Questions regarding the personal experiences, emotions and preferences, with a strong activation on questions about Gemma itself.
- **14350**: False information detection: Detects when the user is providing false information.
- **62019**: False information detection: Activates on user inputs containing incorrect information, similar to Latent 14350, but activates more strongly on template tokens.
- **58070**: Missing information detection: Activates on user inputs containing missing information.
- **54087**: Rewriting requests: Activates when the model should rewrite or paraphrase something.
- **50586**: Joke detection: Activates after jokes or humorous content.
- **69447**: Response length measurement: measures requested response length, with highest activation on a request for a paragraph.
- **10925**: Summarization requests: Activates when the user requests a summary.
- **6583**: Knowledge boundaries: Activates when the model is missing access to information.
- **4622**: Information detail detection: Activates on requests for detailed information.

### Enjoy your dashboard!

In [5]:
from tools.utils import offline_dashboard
off_dashboard = offline_dashboard(crosscoder)

# Online Feature Dashboard

In [6]:
import sys
sys.path.append("..")
from tools.utils import online_dashboard
online_dashboard(crosscoder).display()

# Inference demo

In [ ]:
from nnterp import load_model
from nnterp.nnsight_utils import get_layer_output, get_layer
from tools.utils import load_dictionary_model

base_model = load_model("google/gemma-2-2b", torch_dtype=th.bfloat16)
chat_model = load_model("google/gemma-2-2b-it", torch_dtype=th.bfloat16)
layer = 13
cc_device = "cuda:0" if th.cuda.is_available() else "cpu"
crosscoder = load_dictionary_model(crosscoder).to(cc_device)
sample_conv = [
    {"role": "user", "content": "Hello, how are you?"},
    {"role": "assistant", "content": "I am fine, thank you!"},
]
toks = chat_model.tokenizer.apply_chat_template(sample_conv, return_tensors="pt")
with base_model.trace(toks):
    base_acts = get_layer_output(base_model, layer).to(cc_device).save()
    get_layer(base_model, layer).output.stop()
with chat_model.trace(toks):
    chat_acts = get_layer_output(chat_model, layer).to(cc_device).save()
    get_layer(chat_model, layer).output.stop()

cc_input = th.stack(
    [
        base_acts.reshape(-1, base_acts.shape[-1]).to(cc_device),
        chat_acts.reshape(-1, chat_acts.shape[-1]).to(cc_device),
    ],
    dim=1,
).float()
print(cc_input.shape)  # (b * seq_len, 2, d)

cc_output = crosscoder(cc_input)

torch.Size([24, 2, 2304])
